In [1]:
import numpy as np
# import impl.neuralnet as nn
from impl.solver import *
import sys

In [2]:
n_iter = 1000 # number of epochs
alpha = 1e-3
mb_size = 64 # minibatch size usually compatible to the Cache/RAM size
n_experiment = 1
reg = 1e-5
print_after = 100
p_dropout = 0.8 # dropout/keep_prob
loss = 'cross_ent'
nonlin = 'relu'
solver = 'sgd'

In [3]:
# import hipsternet.input_data as input_data  # NOT used for MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('data/MNIST_data/', one_hot=False)
X_train, y_train = mnist.train.images, mnist.train.labels
X_val, y_val = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels
y_test.shape, y_val.shape, y_train.shape

Extracting data/MNIST_data/train-images-idx3-ubyte.gz
Extracting data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting data/MNIST_data/t10k-labels-idx1-ubyte.gz


((10000,), (5000,), (55000,))

In [4]:
M, D, C = X_train.shape[0], X_train.shape[1], y_train.max() + 1
M, D, C

(55000, 784, 10)

In [6]:
import impl.utils as utils
X_train, X_val, X_test = utils.prepro(X_train, X_val, X_test)
X_train.shape, X_val.shape, X_test.shape

((55000, 784), (5000, 784), (10000, 784))

In [7]:
# # if net_type == 'cnn':
# img_shape = (1, 28, 28)
# X_train = X_train.reshape(-1, *img_shape)
# X_val = X_val.reshape(-1, *img_shape)
# X_test = X_test.reshape(-1, *img_shape)
# X_train.shape, X_val.shape, X_test.shape
# # ((55000, 1, 28, 28), (5000, 1, 28, 28), (10000, 1, 28, 28))
# # In [19]:

In [8]:
solvers = dict(
    sgd=sgd,
    momentum=momentum,
    nesterov=nesterov,
    adagrad=adagrad,
    rmsprop=rmsprop,
    adam=adam
)
solvers

{'adagrad': <function impl.solver.adagrad>,
 'adam': <function impl.solver.adam>,
 'momentum': <function impl.solver.momentum>,
 'nesterov': <function impl.solver.nesterov>,
 'rmsprop': <function impl.solver.rmsprop>,
 'sgd': <function impl.solver.sgd>}

In [9]:
solver_fun = solvers[solver] # solver functions
accs = np.zeros(n_experiment)
solver_fun, accs

(<function impl.solver.sgd>, array([ 0.]))

In [10]:
print()
print('Experimenting on {}'.format(solver))
print()


Experimenting on sgd



In [11]:
import impl.FFNN as ffnn # Feed Forward Neural Net

for k in range(n_experiment):
#     print('PReLU as the activation function')
    print('ReLU as the activation function')
#     print('Softplus as the activation function')
#     print('Sigmoid as the activation function')
#     print('ELU as the activation function')
#     print('Neural_Biased_Fully_Parametric_leaky_ReLU as the activation function')
#     print('Biased_Fully_Parametric_ReLU as the activation function')
#     print('Fully_Parametric_ReLU as the activation function')
#     print('P_Leaky ReLU as the activation function')
#     print('Leaky ReLU as the activation function')
#     print('Noisy ReLU as the activation function')
#     print('Tanh as the activation function')
#     print('Integral Tanh as the activation function')
    print('Experiment-{}'.format(k + 1))

    # net = nn.FeedForwardNet(D, C, H=128, lam=reg, p_dropout=p_dropout, loss=loss, nonlin=nonlin)
    net = ffnn.FFNN(D, C, H=128, lam=reg, p_dropout=p_dropout, loss=loss, nonlin=nonlin)
    #     net = nn.ConvNet(D=10, C=C, H=128) # Original one
#     net = ConvNet_(C=C, D=10, H=128) # Mine
    

    net = solver_fun(nn=net, X_train=X_train, y_train=y_train, val_set=(X_val, y_val), 
                     mb_size=mb_size, alpha=alpha, n_iter=n_iter, print_after=print_after)

    y_pred = net.predict(X_test)
    accs[k] = np.mean(y_pred == y_test)

print()
print('Test Mean accuracy: {:.4f}, std: {:.4f}'.format(accs.mean(), accs.std()))

ReLU as the activation function
Experiment-1
Iter-100 loss: 2.3149 validation: 0.197600
Iter-200 loss: 2.1908 validation: 0.356400
Iter-300 loss: 2.0548 validation: 0.481600
Iter-400 loss: 1.8680 validation: 0.565400
Iter-500 loss: 1.5860 validation: 0.611000
Iter-600 loss: 1.3055 validation: 0.657400
Iter-700 loss: 1.3702 validation: 0.686600
Iter-800 loss: 1.5058 validation: 0.706400
Iter-900 loss: 1.2148 validation: 0.726800
Iter-1000 loss: 1.1577 validation: 0.744400

Test Mean accuracy: 0.7421, std: 0.0000


# Future work: -----------------